In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchsummary import summary

In [3]:
from tqdm import tqdm
from MNISTData import MNISTData
from AutoEncoder import AutoEncoder

In [4]:
ENCODING_SIZE = 16
NUM_QUBITS = 10

In [5]:
dev = qml.device('default.qubit', wires=NUM_QUBITS)

In [6]:
from pennylane.ops import RX, RY, CNOT

# x will be a length 16 vector that represents
# the encoding of a MNIST image
@qml.qnode(dev, interface='torch')
def circuit(x, thetas):
    for i in range(NUM_QUBITS):
        RX(x[i], wires=i)
    for i in range(NUM_QUBITS, ENCODING_SIZE):
        RY(x[i], wires=(i - NUM_QUBITS))
    for i in range(NUM_QUBITS - 1):
        CNOT(wires=[i, i+1])
    for i in range(NUM_QUBITS):
        RX(thetas[i], wires=i)
    for i in range(NUM_QUBITS, 2 * NUM_QUBITS):
        RY(thetas[i], wires=(i - NUM_QUBITS))
    return tuple(qml.expval.PauliZ(wires=i) for i in range(NUM_QUBITS))

In [7]:
# example input
print(circuit([np.pi/5]*(ENCODING_SIZE), [np.pi]*(2 * NUM_QUBITS)))

tensor([0.6545, 0.4284, 0.2804, 0.1835, 0.1201, 0.0786, 0.0636, 0.0515, 0.0416,
        0.0337], dtype=torch.float64)


In [8]:
loss = nn.CrossEntropyLoss()
def cost(X, thetas, actual_labels):
    ret_val = torch.tensor(0., requires_grad=True).float()
    for i in range(X.size(0)):
        y = circuit(X[0], thetas)
        ret_val = ret_val + loss(torch.unsqueeze(y, dim=0).float(), torch.unsqueeze(actual_labels[i], dim=0))
    return ret_val.float()

At this point, we have to get our (encoded) images so that we may actually start training

In [9]:
data = MNISTData()
train_loader = data.get_train_loader()
test_loader = data.get_test_loader()

In [10]:
load_from = "./autoencoder_models/1558553790/ae.pt"

In [11]:
ae = AutoEncoder()
ae.load_state_dict(torch.load(load_from))
encoder = ae.encoder
for child in encoder.children():
    for param in child.parameters():
        param.requires_grad = False

In [12]:
summary(encoder, input_size=(28 * 28,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 256]         200,960
              ReLU-2                  [-1, 256]               0
            Linear-3                   [-1, 64]          16,448
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 16]           1,040
              ReLU-6                   [-1, 16]               0
Total params: 218,448
Trainable params: 0
Non-trainable params: 218,448
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.83
Estimated Total Size (MB): 0.84
----------------------------------------------------------------


In [13]:
thetas = Variable(torch.tensor(2 * np.random.rand(2 * NUM_QUBITS) - 1), requires_grad=True).long()
# W = Variable(torch.tensor(2 * np.random.rand(NUM_QUBITS, 10) - 1), requires_grad=True).long()

In [14]:
optimizer = optim.SGD([thetas], lr=.01, momentum=.9)

In [15]:
enums = enumerate(train_loader, 0)
i, data = next(enums)
inputs, labels = data
print(inputs[0].size())
print(len(labels))
# this is the transformation that you need to do in order
# to pass it to the encoder. it is (1, -1) since the first number
# is the batch size, which in our case is 1
print(encoder(inputs[0].view(1, -1)))
print(encoder(inputs[0].view(1, -1)).size())

torch.Size([1, 28, 28])
128
tensor([[25.4003, 13.1641, 16.0061,  8.9862,  9.2343, 26.6138, 15.6111,  2.8361,
         15.4782, 27.8355,  0.0000, 14.6405,  0.0000,  3.1332,  5.1923, 13.7618]])
torch.Size([1, 16])


In [16]:
# for an explanation of this cell, look at
# https://pytorch.org/docs/stable/optim.html
# as well as 
# https://pennylane.readthedocs.io/en/latest/code/interfaces/torch.html

# as of right now, training takes about three seconds for one example
# at that rate, it would take 50 hours for one epoch
for epoch in range(25):
    for i, data in tqdm(enumerate(train_loader)):
        inputs, labels = data
        def closure():
            optimizer.zero_grad()
            X = encoder(inputs.view(len(labels), -1))
            loss = cost(X, thetas, labels)
            loss.backward()
            return loss
        optimizer.step(closure)
        print("done with batch %d" % i)

1it [01:58, 118.25s/it]

done with batch 0


2it [04:04, 120.55s/it]

done with batch 1


3it [06:02, 119.96s/it]

done with batch 2


KeyboardInterrupt: 